## Link Prediction

* Given an observed network
    * identify missing links
    * predict potential links

### Heuristic Methods

* Common neighbors: number of common neighbors between two nodes

* Jaccard similarity: common neighbors normalized by the size of the joint neighborhood
$$ Jaccard(i,j)=\frac{|\Gamma(i)\cap\Gamma(j)|}{|\Gamma(i)\cup\Gamma(j)|} $$

* Adamic/Adar: number of weighted common neighbors; each neighbors is discounted by its degree.
$$ AdamicAdar(i,j)=\sum_{z\in\Gamma(i)\cap\Gamma(j)}\frac{1}{\log|\Gamma(z)|} $$

* Distance: length of shortest paths.

* Kats similarity: number of paths between two nodes but the contribution from long paths is discounted.
$$ Katz(i,j)=\sum_{l=1}^{\infty}\beta^l~|paths_{i,j}^l|~~~~or~~~~Katz=(I-\beta A)^{-1}-I$$

* Rooted Pagerank: the similarty between nodes $i$ and $j$ is the stationary probability of visiting $j$ of a random walk that starts from $i$ and returns to $i$ with probability $1-\beta$ in every step.
$$ Pagerank=(1-\beta)(I-\beta D^{-1}A)^{-1} $$

### Collaborative Filtering

* A popular method in recommender systems.
* Mostly used for bi-partite networks, e.g., users and items, to predict
    * which movie a user will watch, what music a user will like, and so on. 
<img src='Images/collaborativefiltering.png'>

#### Key idea

* Individuals’ preferences can be predicted from similar others’
    * If my purchase history is similar to yours, then I will probably buy what you buy next
    * If I bought product A and A is often bought together with product B, then I will probably buy B next
    
    

The idea behind collaborative filtering is that individuals’ preferences can be predicted from similar others’ given large-scale population data. For concreteness, let’s consider consumers and products in e-commerce. The idea behind collaborative filtering is the following: we can probably make a good guess on what a consumer will buy next by considering other consumers that have similar purchase histories as this target consumer; on the other hand, for a consumer who has bought product A, I should probably recommend her product B, the product whose buyers have the largest overlap with A’s (e.g., the "people who bought ... also bought ..." section seen on most e-commerce platforms).

#### Latent factor model
* Each individual is associated with a few latent factors, and so is each product
* Interactions between the individuals and the products are determined by the similarity between the individuals and the products in terms of the latent factors.
* The latent factors are unknown and to be estimated

A widely used collaborative filtering technique is the latent factor model. It assumes that each individual is associated with a few latent factors, and so is each product; in other words, the individuals and the products are mapped to a same latent space. And then the (valued) interactions between the individuals and the products - e.g., ratings, likes and dislikes, and purchases - are determined by the distances between the individuals and the products in the latent space. However, the latent factors and the positions are unknown and they are usually estimated by the matrix factorization method.

#### Model

* Each user $i$ is associated with a latent vector $P_i=(P_{i1},P_{i2},\cdots,P_{iK})$  
* Each item $j$ is associated with a latent vector $Q_j=(Q_{j1},Q_{j2},\cdots,Q_{jK})$ 
* The weighted edge $A_{ij}$ between $i$ and $j$ is assumed to be approximately $P_i\cdot Q_j$. 
* In matrix form, $A\approx P Q^T$

#### Estimation

* Given the observed $A_{ij}$, the goal is to find the latent vectors that minimize the following cost function:
$$
\sum_{(i,j)\in E} (A_{ij} - \sum_{k=1}^K P_{ik}Q_{jk})^2  + \beta(||P||^2 + ||Q||^2),
$$
    * $E$ is the set of $(i,j)$ pairs whose $A_{ij}$ are known. 
* This estimation is approximately a factorization of $A$, that is, $A\approx P Q^T$.

### Example: Ratings on Video Games

The data is about the world's most popular PC Gaming hub, Steam, which covers over 6,000 video games and a community of millions of users or gamers. The data is organized as a table with 3 columns (variables): user ID, game ID, and rating; and each row (observation) represent the rating from the user on the game.  

In total the data has 43,726 observations from 8,203 users and 284 games. The numbers are smaller than the size of the Steam community because we only include games that have been played by at least 50 users and users who have played at least two hours.

First, load data into R

In [1]:
dat = read.csv('dataset-steam-2017.csv')

To obtain a fair evaluation of the method, split the data randomly into a training subset and a test subset:

In [2]:
index = sample(seq_len(nrow(dat)), size = floor(0.9*nrow(dat)))	
train = dat[index,]
test = dat[-index,]

The collaborative filtering method is implemented by the package "recosystem", so install it before it can loaded:

In [3]:
install.packages('recosystem')
library('recosystem')

Next, we need to turn the training set into the format required by the recosystem package:

In [4]:
rec_train = data_memory(user_index = train$user_id, 
                       item_index = train$item_id, 
                        rating=train$rating,
                        index1 = TRUE) 

After that we can fit the latent factor model to the training set:

In [5]:
rec = Reco()
res = rec$train(rec_train, opts = list(ndim=10, verbose=TRUE))

iter      tr_rmse          obj
   0       0.3715   8.4745e+03
   1       0.3141   7.0554e+03
   2       0.3004   6.7184e+03
   3       0.2939   6.6016e+03
   4       0.2894   6.5112e+03
   5       0.2862   6.4569e+03
   6       0.2839   6.4081e+03
   7       0.2823   6.4071e+03
   8       0.2805   6.3493e+03
   9       0.2795   6.3336e+03
  10       0.2787   6.3294e+03
  11       0.2779   6.3163e+03
  12       0.2770   6.2895e+03
  13       0.2765   6.2778e+03
  14       0.2762   6.2915e+03
  15       0.2754   6.2547e+03
  16       0.2754   6.2721e+03
  17       0.2747   6.2554e+03
  18       0.2746   6.2585e+03
  19       0.2744   6.2647e+03


where "ndim"" in the second line specifies the number of latent factors, which is set to 10 in this example.

Finally, we assess the predictive performance of the method on the test set. But first we still need to convert the test set to the format required by the package:

In [6]:
rec_test = data_memory(user_index = test$user_id, 
                      item_index = test$item_id,
                      index1 = TRUE)

Then, the mean squared error of the predictions of ratings can be calculated as:

In [7]:
pred = rec$predict(rec_test, out_pred = out_memory())
sum((test$rating-pred)^2) / length(pred)

[1] 0.09469553

### Exponential Random Graph Model (ERGM)

* A probability model for networks
* Similar to logistic regression in predicting edges
* Can handle attributes of nodes and edges
* Can handle network structures such as transitivity

The expoential random graph model is for general link predictions, especially when nodes have attributes. It is a parametric model for the probability distribution of networks. It can be viewed as a generalization of the Logistic regression to predict edges: for every pair of nodes, the outcome is whether or not there is an edge between the two nodes and predictors are any attributes of the nodes. However, ERGM is more general and models the probability of the network as a whole instead of modelling every pair of nodes. Consequently, it even allows using any statistics of a network including global properties such as edge density and global transitivity. Most importantly, by modelling the probability of a whole network ERGM considers the correlation structure between pairs, which is overlooked in the Logistic regression model where the pairs are assumed to be independent. 

#### Logistic regression model

* Outcome variable $A_{ij}$
* Predictors: attributes of nodes $i$ and $j$, $x(i)$, $x(j)$, and $x(i,j)$
* Logistic regression
$$
A_{ij} \sim c_0 + c_1 x(i) + c_2 x(j) + c_3 x(i,j)
$$
* Assumption: the pairs are independent from each other

#### Exponential Random Graph Model

* Let $A$ be a random adjacency matrix and $a$ be a realization of $A$. 
* Let $x(a)=(x_1,…,x_p)$ to be the vector of statistics of the network $a$ that we are interested in, e.g.,   $x(a)=$(number of triangles, number of edges between people of same race, number of edges between people of similar grades).
* ERGM specifies a probability model on $A$ as the following:
$$
P(A=a)=\frac{\exp(c \cdot x(a))}{k(c)},
$$
    * where $c$ is a vector of coefficients corresponding the statistics $x(a)$.

#### Estimation

* Given an observed network and its adjacency matrix $a$, our goal is to find the coefficients $c$ that maximize the probability of observing $a$ based on the model. 
* But we only have one observation of the network, how to estimate

However, most of the time we only observe one or two snapshots of the network under study; how can we estimate this model with only one data point? This issue is addressed by advanced MCMC techniques, but the estimation process is quite complicated and is hence beyond the scope of this course. The rough idea behind it is similar to the maximal likelihood estimation for logistic regressions. Finally, the estimates of the coefficients and their statistical significance tell us how much each statistic in the vector $x(a)$ contributes to the formation of edges and the network. 

* The ERGM model can be written as
$$
logit(A_{ij}=1|a_{ij}^c) = c \cdot(x(a_{ij}^+) - x(a_{ij}^- )).
$$
    * $a_{ij}^c$ means all the pairs of nodes except for the $(i,j)$ pair
    * $a_{ij}^+$ and $a_{ij}^-$ are the same as $a$ but with the entry $a_{ij}$ set to 1 and 0, respectively
* Estimation is done by advanced MCMC techniques.

* Model
$$
P(A=a)=\frac{\exp(c \cdot x(a))}{k(c)},
$$
* The coefficients and their statistical significance tell us how much each statistic in the vector $x(a)$ contributes to the formation of edges.

### Example: UK Faculty Network

The data is about a social network consisting of 81 nodes with each representing a faculty member of a UK university, and the edges of the network represent friendships between the faculty members. There are 817 undirected, unweighted edges in total. Here we are interested in predicting edges based on the department affiliations of the faculty members.

First, load the network data into R

In [8]:
nodes = read.csv('dataset-ukfaculty-2008-nodes.csv') 
edges = read.csv('dataset-ukfaculty-2008-edges.csv')

The ERGM is included in the "statnet" package. We need to load the  package in order to use it.

In [9]:
install.packages('statnet')
library('statnet')

Loading required package: tergm
Loading required package: statnet.common

Attaching package: ‘statnet.common’

The following object is masked from ‘package:base’:

    order

Loading required package: ergm
Loading required package: network
network: Classes for Relational Data
Version 1.13.0.1 created on 2015-08-31.
copyright (c) 2005, Carter T. Butts, University of California-Irvine
                    Mark S. Handcock, University of California -- Los Angeles
                    David R. Hunter, Penn State University
                    Martina Morris, University of Washington
                    Skye Bender-deMoll, University of Washington
 For citation information, type citation("network").
 Type help("network-package") to get started.


ergm: version 3.9.4, created on 2018-08-15
Copyright (c) 2018, Mark S. Handcock, University of California -- Los Angeles
                    David R. Hunter, Penn State University
                    Carter T. Butts, University of California -- Irvin

               Installed  ReposVer Built  
ergm           "3.9.4"    "3.10.4" "3.4.4"
ergm.count     "3.2.2"    "3.4.0"  "3.4.0"
network        "1.13.0.1" "1.15"   "3.4.4"
networkDynamic "0.9.0"    "0.10.0" "3.4.0"
tergm          "3.4.1"    "3.6.1"  "3.4.1"
tsna           "0.2.0"    "0.3.0"  "3.4.0"


Restart R and use "statnet::update_statnet()" to get the updates.


Next, we can turn the node and edge tables into a network object by the following command:

In [10]:
colnames(edges)=c('head','tail')
G = network(edges, directed = F, matrix.type='edgelist')

Then we need to set the attributes of the nodes, which can done as the following:

In [11]:
G %v% 'dept' = nodes$dept

* Finally, we fit the ERGM to the network with two network statistics
    * The number of edges  
    * The number of edges between nodes from the same department

In [12]:
model = ergm(G ~ edges + nodematch('dept'))

Starting maximum pseudolikelihood estimation (MPLE):
Evaluating the predictor and response matrix.
Maximizing the pseudolikelihood.
Finished MPLE.
Stopping at the initial estimate.
Evaluating log-likelihood at the estimate. 


The estimation results can be checked as with a regression model:

In [13]:
summary(model)


Summary of model fit

Formula:   G ~ edges + nodematch("dept")

Iterations:  6 out of 20 

Monte Carlo MLE Results:
               Estimate Std. Error MCMC % z value Pr(>|z|)    
edges          -2.83855    0.09353      0  -30.35   <1e-04 ***
nodematch.dept  2.57248    0.11235      0   22.90   <1e-04 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

     Null Deviance: 4492  on 3240  degrees of freedom
 Residual Deviance: 2374  on 3238  degrees of freedom
 
AIC: 2378    BIC: 2391    (Smaller is better.) 

### Excercise: How to predict?

* Use simulate to simulate networks
* Use the simulated network to measure the probability of each potential edge

In [17]:
Ahat=matrix(0, 81, 81)

for (i in 1:10) {
  a = simulate(model)
  Ahat = Ahat + as.matrix(a)
}

Ahat=Ahat/10